In [1]:
import pandas as pd
import numpy as np


In [2]:
# Load the Book-Crossing dataset
books_df = pd.read_csv('../data/Books.csv', delimiter=';', low_memory=False)
ratings_df = pd.read_csv('../data/Ratings.csv', delimiter=';')
users_df = pd.read_csv('../data/Users.csv', delimiter=';')

/var/folders/s0/4m_t_82936ggv17n0g0t7_dr0000gn/T/ipykernel_3953/1329463720.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users_df = pd.read_csv('../data/Users.csv', delimiter=';')


In [5]:
# Preprocessing
ratings_df = ratings_df[ratings_df['Rating'] > 0]  # Remove zero ratings
ratings_df['User-ID'] = ratings_df['User-ID'].astype(int)
ratings_df['ISBN'] = ratings_df['ISBN'].astype(str)
ratings_df['Rating'] = ratings_df['Rating'].astype(int)

# Merge the data to get book titles along with ratings
data = pd.merge(ratings_df, books_df[['ISBN', 'Title']], on='ISBN')

In [6]:
class EpsilonGreedyMAB:
    def __init__(self, epsilon=0.1):
        self.epsilon = epsilon
        self.book_rewards = {}
        self.book_counts = {}

    def select_book(self, book_list):
        if np.random.rand() < self.epsilon:
            # Explore: select a random book
            return np.random.choice(book_list)
        else:
            # Exploit: select the book with the highest average reward
            return max(book_list, key=lambda x: self.book_rewards.get(x, 0) / (self.book_counts.get(x, 1)))

    def update(self, book, reward):
        if book in self.book_rewards:
            self.book_rewards[book] += reward
            self.book_counts[book] += 1
        else:
            self.book_rewards[book] = reward
            self.book_counts[book] = 1


In [10]:
# Initialize the MAB algorithm
mab = EpsilonGreedyMAB(epsilon=0.1)



In [11]:
# Simulate user interactions and recommendations
def simulate_user_interaction(user_id, mab):
    user_books = data[data['User-ID'] == user_id]['Title'].unique()
    if len(user_books) == 0:
        return None
    
    recommended_book = mab.select_book(user_books)
    actual_rating = data[(data['User-ID'] == user_id) & (data['Title'] == recommended_book)]['Rating'].values[0]
    
    # Update the MAB with the observed reward
    mab.update(recommended_book, actual_rating)

    return recommended_book, actual_rating

In [12]:
# Simulate recommendations for a subset of users
user_ids = data['User-ID'].unique()[:1000]  # Consider first 100 users for simplicity
results = []

for user_id in user_ids:
    recommended_book, actual_rating = simulate_user_interaction(user_id, mab)
    if recommended_book:
        results.append((user_id, recommended_book, actual_rating))

# Display the results
results_df = pd.DataFrame(results, columns=['User-ID', 'Recommended Book', 'Actual Rating'])
print(results_df.head())


   User-ID                             Recommended Book  Actual Rating
0   276726                             Rites of Passage              5
1   276729                               Help!: Level 1              3
2   276744                              A Painted House              7
3   276747                     Little Altars Everywhere              9
4   276748  Apricots on the Nile: A Memoir with Recipes              6


In [17]:
results_df['Actual Rating'].value_counts() / len(results_df) * 100

Actual Rating
8     25.6
7     20.2
10    16.6
9     13.6
5     10.7
6      7.9
4      2.5
3      1.5
1      0.8
2      0.6
Name: count, dtype: float64

In [16]:
ratings_df['Rating'].value_counts() / len(ratings_df) * 100

Rating
8     23.920437
10    18.126644
7     17.630185
9     15.574249
5     11.754072
6      8.514288
4      2.053169
3      1.382615
2      0.636197
1      0.408144
Name: count, dtype: float64